In [1]:
import modis
import ee
import datetime as dt
import pfast
import geemap as emap

Enter verification code: 4/1AWtgzh6iC5kN5lmkrZlOdfpkR3d_NEGq6oA4VKo6L5etDRBmqWPGz3No2jA

Successfully saved authorization token.


In [2]:
Map=emap.Map()
# Load the study area
vn=ee.FeatureCollection("users/miketu72/VN_Map")

- **This notebook aims to extract cloud cover percent over the period of 2015 to 2021.**

Each image has one band called quality band and Landsat 8 named it QA_PIXEL. So the idea is to access bits 8-9 and get the high cloud confidence and assigned it to 1. Finally, we can map over all images in the collection and calculate the percentage of cloud confidence per pixel.

# 1. Landsat Collection

This example demonstrates the use of the Landsat 8 Collection 2, Level 2 `QA_PIXEL` band (CFMask) to mask unwanted pixels.

In [3]:
ls8=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(vn).filterDate("2016","2022")

In [4]:
def ls8_cloud_mask(img):
    """ Mask cloud and cloud shadow pixels from Landsat 8 Collection 2 Surface Reflectance LANDSAT/LC08/C02/T1_L2
    
        Args:
            - img (ee.Image): An input Landsat 8 SR
        
        Returns:
            - An image (ee.Image): The image without cloud and cloud shadow pixels. 
    """
    
    qa=img.select('QA_PIXEL')
    
    cloud_bit_mask=ee.Number(2).pow(5).int()
    shadow_bit_mask=ee.Number(2).pow(3).int()
    
    cloud_mask=qa.bitwiseAnd(cloud_bit_mask).eq(0)
    shadow_mask=qa.bitwiseAnd(shadow_bit_mask).eq(0)
    
    mask=cloud_mask.And(shadow_mask)
    return img.updateMask(mask)

In [5]:
def cloud_pixel(img):
    """
        Converts masked values (None) in an ee.Image to be 1 and all other values to be 0.
    
        Args:
            - img (ee.Image): The input image.
    
        Returns:
            - An image (ee.Image): The image with masked values converted to 1 and all other values converted to None.
    """
    # Create a binary mask from the original image
    binary_mask = img.mask().Not()
    
    # Use the binary mask to convert masked values in the original image to 1
    out_bin = binary_mask.where(binary_mask, 1).where(img.mask(), 0)
    out_clip=out_bin.clip(img.geometry())
    
    out_mask=out_clip.eq(1)
    out_img=out_clip.updateMask(out_mask)
    
    # Set the spatial extent of the output image to be the same as the input image
    return out_img.rename("cloud_band")

In [6]:
def cloud_free_pixel(img):
    """
        Converts all non-masked values in an ee.Image to be equal to 1 and all masked values to be None.
    
        Args:
            - img (ee.Image): The input image.
    
        Returns:
            - ee.Image: The converted image.
    """
    out_img=img.mask()
    
    return out_img.clip(img.geometry()).rename("cloudless_band")

In [8]:
# Get cloud free images
cloudless_col_ls8=ls8.map(ls8_cloud_mask).select("SR_B3")

- **The percentage of monthly cloud-free pixels from 2016 to 2022**

In [9]:
# Convert cloud pixels to be 1 and all other values to be None
cloud_free_pixel_col=cloudless_col_ls8.map(cloud_free_pixel)

In [16]:
month_list=ee.List.sequence(1,12)

def monthly_free_cloud(month):
    sub_col=cloud_free_pixel_col.filter(ee.Filter.calendarRange(month,month,"month"))
    index=ee.Number(month)
    sub_col_percent=sub_col.sum().divide(sub_col.size()).multiply(100).set({"month":index})
    return sub_col_percent.rename("cloud_free").clip(vn)

monthly_cloudless_percent=ee.ImageCollection.fromImages(month_list.map(monthly_free_cloud))

- Exceed user limit and mannual operations

In [17]:
from collections import namedtuple

cloudless_percent_list=[]

for i in range(1,13):
    thang=cloud_free_pixel_col.filter(ee.Filter.calendarRange(i,i,"month"))
    size_col=thang.size()
    out_img=thang.sum().divide(size_col).multiply(100).clip(vn)
    cloudless_percent_list.append((str(i),out_img))

In [19]:
Map=emap.Map()
Map.setCenter(106.8,16.28,6)
# vis_params = {'bands': ['cloudless_band'], 'palette': ['5e4fa2', ' 3683bb', ' 5db7a9', ' 98d6a4', ' d1ec9c', ' f4faad', ' fff1a7', ' fece7c', ' fb9c59', ' ee6445', ' d0384e', ' 9e0142'], 'min': 0.0, 'max': 100.0}
vis_params = {'bands': ['cloud_free'], 'palette': ['5e4fa2', ' 3683bb', ' 5db7a9', ' 98d6a4', ' d1ec9c', ' f4faad', ' fff1a7', ' fece7c', ' fb9c59', ' ee6445', ' d0384e', ' 9e0142'], 'min': 0, 'max':9}
Map.addLayer(monthly_cloudless_percent.first(),vis_params,"Landsat Cloud-free Percent")
Map

Map(center=[16.28, 106.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…